<a href="https://colab.research.google.com/github/NikithaBangera/COMP6721_Artificial_Intelligence/blob/main/Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print ("Yashika here!")
Print ("Yashika here 2")


Yashika here!


# New Section

In [ ]:
print("Nikitha")

Nikitha
